# Implementation of the Branch and Price
## **Model : One customer, multiple suppliers, no time windows**

## 1. Set Packing | Partitioning | Covering formulation


Here are some notations I'll use :
- $\Omega$ is the set of all **feasible** routes.
- $(c_{r})_{r\in\Omega}$ is the cost of route $r$.
- $\nu_{\ell, r}$ = 1 if location $\ell$ is visited in route $r$, 0 otherwise.
- $a_{u,v,r}$ = 1 if in route $r$, an arc from $u$ to $v$ is used, 0 otherwise.
- $p_{c,r}$ is the amount of paper in the truck following route $r$, in order to deliver customer $c$
- $p_{s,r}$ is the amount of paper in the truck following route $r$ that has been picked up at supplier $s$. Equals 0 if supplier $s$ is not visited in route $r$


We can deduce the following relations :
- $c_{r} = \sum_{u,v\in\mathcal{L}}d_{u,v}a_{u,v,r}$
- $\nu_{\ell, r} = \sum_{u\in\mathcal{L}}a_{u,\ell,r}$
- $p_{c,r} = p_{c}\nu_{c, r}$
- $p_{s,r} = p_{s}\nu_{s, r}$

Let $(\theta_{r})_{r\in\Omega}\in\mathbb{N}$ be the number of times route $r$ is used. ($\theta_{r}\geq2$ yields infeasible solutions, but adding boolean-like constraints add variables in the dual.)

The following formulation applies :
$$
\begin{aligned}
\min &\sum_{r\in\Omega}c_{r}\theta_{r} \\
\textsf{such that :} &\sum_{r\in\Omega}\nu_{c,r}\theta_{r} = 1, \forall{c}\in\mathcal{C} \\
&\sum_{r\in\Omega}\nu_{s,r}\theta_{r} \leq 1, \forall{s}\in\mathcal{S} \\
&\theta_{r}\sum_{c\in\mathcal{C}}p_{c}\nu_{c,r} \leq q, \forall{r}\in\Omega\\
&\theta_{r}\sum_{s\in\mathcal{S}}p_{s}\nu_{s,r} \leq q, \forall{r}\in\Omega\\
&\sum_{r\in\Omega\atop s\in\mathcal{S}}p_{s}\nu_{s,r}\theta_{r}\geq m\\
&\theta_{r}\geq0, \forall{r}\in\Omega\\
\end{aligned}
$$

## 2. Towards the pricing
### 2.1 - Dual of the linear relaxation 

$$
\begin{aligned}
\max &\sum_{c\in\mathcal{C}}\alpha_{c} + \sum_{s\in\mathcal{S}}\beta_{s} + \sum_{r\in\Omega}q(\delta_{r} + \gamma_{r}) + m\ \varepsilon\\
\textsf{such that :}&\\
&\sum_{c\in\mathcal{C}}(\alpha_{c}+p_{c}\delta_{r})\nu_{c,r} + \sum_{s\in\mathcal{S}}\nu_{s,r}(\beta_{s}+(\varepsilon + \gamma_{r})p_{s})\leq c_{r}, \forall{r}\in\Omega \\
&\alpha_{c} \in\mathbb{R}, \forall{c}\in\mathcal{C}\\
&\beta_{s}\leq0, \forall{s}\in\mathcal{S}\\
&\delta_{r}\leq0, \forall{r}\in\Omega\\
&\gamma_{r}\leq0, \forall{r}\in\Omega\\
&\varepsilon\geq0
\end{aligned}
$$

### 2.1 - Reduced cost of a given route :
We are looking for a route $r\in\Omega\setminus\Omega'$ that has the minimum reduced cost :
$$\min_{r\in\Omega\setminus\Omega'}\bar{c_{r}} = c_{r} - \sum_{c\in\mathcal{C}}(\alpha_{c}+p_{c}\delta_{r})\nu_{c,r} - \sum_{s\in\mathcal{S}}(\beta_{s}+(\varepsilon + \gamma_{r})p_{s})\nu_{s,r}$$

### 2.2 - Pricing problem
Rather than enumerating all routes, we use a two index formulation, and propose the following formulation of the pricing problem :

Here are the constraints that apply to $(y_{u, v})_{u,v\in\mathcal{L}}$

1. The following variables will not be generated / forced to 0
    - No single loops : $y_{u,u}, \forall{u}\in\mathcal{L} = 0$
    - Customer visited directly from the depot : $y_{\ell, c} = 0, \forall\ell\in\mathcal{C}\cup\mathcal{S}, \forall{c}\in\mathcal{C}$
    - No direct trip to the supplier : $y_{S, s} = 0, \forall{s}\in\mathcal{S}$

2. Balance equation:
    - $\sum_{u\in\mathcal{L}}y_{u, v} = \sum_{w\in\mathcal{L}}y_{v,w}, \forall{v}\in\mathcal{L}$

3. Leaving to join one customer:
    - $\sum_{c\in\mathcal{C}}y_{S, c} = 1$

4. A supplier is visited at most once:
    - $\sum_{u\in\mathcal{L}}y_{\ell, s} \leq 1, \forall{s}\in\mathcal{S}$

5. Removal of routes in $\Omega'$:
    - $\sum_{(u, v)\in{r'}}y_{u,v} \leq |r'| - 1, \forall{r'}\in\Omega'$

6. No subtours

Below is the complete formulation :
$$
\begin{aligned}
    \min &\sum_{u,v\in\mathcal{L}}d_{uv}y_{u,v} -\sum_{c\in\mathcal{C}}\alpha_{c}y_{S,c} - \sum_{s\in\mathcal{S}\atop\ell\in\mathcal{L}}(\beta_{s} + \varepsilon p_{s})y_{\ell,s} \\
    \textsf{Not generated during variable creation :} &\left\lbrace\begin{aligned}y_{u,u} = 0, &\forall{u}\in\mathcal{L}\\y_{\ell,c}=0, &\forall{\ell}\in\mathcal{L}\setminus\{S\}\\y_{S, s} = 0, &\forall{s}\in\mathcal{S} \end{aligned}\right.\\
    \textsf{Such that :}&\\
    &\sum_{u\in\mathcal{L}}y_{u,v} = \sum_{y\in\mathcal{L}}y_{v,w}, \forall{v}\in\mathcal{L}\\
    &\sum_{c\in\mathcal{C}}y_{S,c} = 1\\
    &\sum_{\ell\in\mathcal{L}}y_{\ell,s} \leq 1, \forall{s}\in\mathcal{S}\\
    &\sum_{(u, v)\in{r'}}y_{u,v} \leq |r'| - 1, \forall{r'}\in\Omega'\\
    &\textsf{No subtour}\\
    &x_{u, v}\in\{0, 1\}, \forall{u, v}\in\mathcal{L}
\end{aligned}
$$

## 3. Increasing the speed of the pricing process
TODO :
1. Allow for subtours after **11** iterations.
2. Write a checker for subtours
3. If has subtours, alternative cost is **1.05** the cost of a 'standard' column